## Importing relevant libraries

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Helper functions

In [ ]:
def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)
    
    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes

    # Get absolute values
    b = np.abs(b)
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_rmsd(list1, list2):
    # Ensure the lists have the same length
    
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length.")
    
    # Compute the RMSD
    rmsd = np.sqrt(np.mean((np.array(list1) - np.array(list2)) ** 2))
    return rmsd

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

## VQD implementation

In [ ]:
# Define the VQD computation with varying ansatz depth
def vqd_resource_estimation(matrix, optimizer, ansatz_depth, seed, zeromode_exact):
    # Function to compute the execution time for different optimizers and ansatz depths (averaged over 10 independent VQD runs in each case)

    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the qubit Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Compute using NumPyMinimumEigensolver for reference
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qub_hamiltonian)
    
    # Define the circuit ansatz with variable depth
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_depth)
    
    # Initial point for the classical optimizer
    seed_value = seed
    np.random.seed(seed_value)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initializing the Estimator, Sampler, and fidelity
    estimator = Estimator()
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler)
    
    # Run the VQE algorithm
    @dataclass
    class VQDLog:
        values: list = None
        parameters: list = None
        
        def __post_init__(self):
            self.values = []
            self.parameters = []

        # Update function to match the expected arguments
        def update(self, count, parameters, mean, _metadata, _extra):
            self.values.append(mean)
            self.parameters.append(parameters)

    log = VQDLog()

    vqd = VQD(estimator,
              fidelity, ansatz, optimizer, k=1, initial_point=initial_point, callback=log.update)
    result = vqd.compute_eigenvalues(qub_hamiltonian)

    # Get the optimizer runtime
    time = result.optimizer_times

    # Estimate the zeromode
    optimal_params = result.optimal_points
    zeromode_points = optimal_params[0]
    final_circuit = ansatz.assign_parameters(zeromode_points)
    zeromode_vqd = Statevector.from_instruction(final_circuit)
    zeromode = zeromode_vqd.data.tolist()

    # Compute the fidelity between the classical and quantum zeromodes
    zeromode_exact = np.array(zeromode_exact).reshape(len(zeromode_exact), 1)
    zeromode = np.array(zeromode).reshape(len(zeromode_exact), 1)
    fidelity = get_similarity(zeromode_exact, zeromode)

    # Get the number of function calls
    num_func_calls = len(log.values)

    return zeromode, fidelity, num_func_calls

## Estimate quantum resources

In [ ]:
def get_resource_estimates_vqd(matrix, zeromode_classic, optimizers, num_qubits, target_fidelity_threshold):
    initial_depth = 1
    max_depth = 10

    # Initialize dictionaries to store results for each optimizer
    min_depths = {}
    zeromodes = {}
    gate_counts = {}
    circuit_depths = {}
    function_calls = {}

    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        print(f"\nRunning VQD for optimizer: {optimizer_name}")

        current_depth = initial_depth
        converged = False  # Flag to check if convergence occurs

        while current_depth <= max_depth:
            fidelities = []
            func_calls = []
            zeromodes_per_run = []

            # Perform multiple independent VQE runs to calculate average fidelity
            for run in range(10):  # Number of independent runs
                seed = run + 1

                # Run VQE, assuming it returns zeromode, fidelity_value, and num_func_calls
                zeromode, fidelity_value, num_func_calls = vqd_resource_estimation(
                    matrix=matrix,
                    optimizer=optimizer,
                    ansatz_depth=current_depth,
                    seed=seed,
                    zeromode_exact=zeromode_classic
                )

                fidelities.append(fidelity_value)
                func_calls.append(num_func_calls)
                zeromodes_per_run.append(zeromode)

            average_fidelity = np.mean(fidelities)
            print(f"Depth {current_depth}: Average fidelity = {average_fidelity}")

            # Check if the average fidelity meets the threshold
            if average_fidelity >= target_fidelity_threshold:
                min_depths[optimizer_name] = current_depth

                # Identify the run with the highest fidelity and its function calls
                max_fidelity_index = np.argmax(fidelities)
                zeromodes[optimizer_name] = zeromodes_per_run[max_fidelity_index]
                function_calls[optimizer_name] = func_calls[max_fidelity_index]
                
                # Calculate gate count and circuit depth for the ansatz at this depth
                ansatz = RealAmplitudes(num_qubits=num_qubits, reps=current_depth)
                decomposed_ansatz = ansatz.decompose()  # Decompose to get actual gate operations
                gate_count_dict = decomposed_ansatz.count_ops()
                total_gates = sum(gate_count_dict.values())
                
                # Store gate counts and circuit depth
                gate_counts[optimizer_name] = total_gates
                circuit_depths[optimizer_name] = decomposed_ansatz.depth()
                
                # Print the gate count, circuit depth, and function calls at optimal depth
                print(f"\nOptimizer: {optimizer_name}")
                print(f"Minimum ansatz depth to meet fidelity threshold: {current_depth}")
                print(f"Total number of gates at optimal depth: {gate_counts[optimizer_name]}")
                print(f"Circuit depth at optimal depth: {circuit_depths[optimizer_name]}")
                print(f"Function calls for the highest fidelity run at optimal depth: {function_calls[optimizer_name]}")
                
                converged = True
                break

            current_depth += 1

        if not converged:
            min_depths[optimizer_name] = "Did not converge"
            zeromodes[optimizer_name] = "Did not converge"
            gate_counts[optimizer_name] = "N/A"
            circuit_depths[optimizer_name] = "N/A"
            function_calls[optimizer_name] = "N/A"
            print(f"{optimizer_name} did not converge within {max_depth} depths.")

    return min_depths, zeromodes, gate_counts, circuit_depths, function_calls